# 필요 모듈 정리

In [ ]:
%pip install selenium
%pip install webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [175]:
import requests
from bs4 import BeautifulSoup
import json
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# jobplanet 페이지 구조

- https://www.jobplanet.co.kr/companies/30139/reviews/삼성전자
    - span[class="rate_point] : 전체 리뷰 평점
    - span[class="txt_point"] : 순서대로 [복지 및 급여, 업무와 삶의 균형, 사내문화, 승진 기회 및 가능성, 경영진, 기업 추천율, ceo 지지율, 성장 가능성]

- https://www.jobplanet.co.kr/companies/30139/benefits/삼성전자
    - span[class="welfare-star__point] : 전체 복지 평점
    - button[class="welfare-overview__num"] : 복지 개수


# jobplanet 기업 고유번호 가져오기
- selenium 이용
- 구글 -> 잡플래닛 + 기업명 검색
- 구글 로봇 정책을 피하기 위해서 10개씩 나누어 jobplanet_corp_code.json 저장

In [13]:
# 기업명을 반복문 돌리기 위한 kospi200 기업 리스트 만들기
with open('top200_name_list.csv', 'r', encoding= 'utf-8') as file :
    kospi_200_corp_list = [corp_name.strip() for corp_name in file.readlines()]

In [77]:
# corp_code 정리를 위한 딕셔너리 생성
job_planet_corp_code = {}
crawling_failed_list = []

In [78]:
for corp in kospi_200_corp_list[190:200] : 
    # 1. 구글 사이트 접속하기
    driver.get('https://www.google.com')

    # 2. 검색창에 커서 놓기
    search_part = driver.find_element(By.CSS_SELECTOR, 'textarea[class="gLFyf"]')

    # 3. 잡플래닛 + 기업명 검색어 입력하기
    search_part.send_keys(f'잡플래닛 {corp}')

    # 4. 구글 검색 아이콘 클릭하기
    driver.find_elements(By.CSS_SELECTOR, 'input[role="button"]')[1].click()

    time.sleep(1)

    # 5. selenium을 통해 동적 html 소스 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    corp_code_htmls = soup.select('a[jsname="UWckNb"]')

    html_tags = ''
    for corp_code in corp_code_htmls :
        html_tags += str(corp_code)
    
    if re.findall('https://www.jobplanet.co.kr/companies/[0-9]+', html_tags) != [] :
        corp_code = int(re.findall('https://www.jobplanet.co.kr/companies/[0-9]+', html_tags)[0].split('/')[-1])
        job_planet_corp_code[corp] = corp_code
    else :
        crawling_failed_list.append(corp)
        print(f'{corp} 회사는 크롤링에 실패했습니다')
        
    
# db 금융지주에서 오류남!

In [79]:
with open('jobplanet_corp_code.json', 'a', encoding='utf-8-sig') as f : 
	json.dump(job_planet_corp_code, f, ensure_ascii=False, indent= '\t')

# 잡플래닛 회사 고유번호를 이용한 잡플래닛 크롤링
- 잡플래닛 자체가 cloudfare로 로봇 크롤링을 막아놨기 때문에 selenium으로 진행
- jobplanet_corp_code.json 불러오기

In [274]:
# 잡플래닛 내부 회사 번호 딕셔너리 불러오기
with open('./jobplanet_corp_code.json', 'r', encoding='utf-8-sig') as file :
    jp_corp_code = json.load(file)
len(jp_corp_code)

200

- 잡플래닛 로그인 하기
    - 셀레니움은 기본적으로 세션 유지가 되기 때문에 로그인 상태를 유지하면서 페이지를 바꿀 수 있다. 

In [281]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# 1. 잡플래닛 로그인 url로 접속하기
driver.get('https://www.jobplanet.co.kr/users/sign_in?_nav=gb')

In [282]:
# 2. 아이디 입력창에 커서 놓기
input_id = driver.find_element(By.CSS_SELECTOR, 'input[id="user_email"]')

# 3. 아이디 입력창에 아이디 입력
input_id.send_keys(f'')

# 4. 비밀번호 입력창에 커서 놓기
input_password = driver.find_element(By.CSS_SELECTOR, 'input[id="user_password"]')

# 5. 비밀번호 입력창에 비밀번호 입력
input_password.send_keys(f'')

# 6. 로그인 버튼 클릭하기
driver.find_element(By.CSS_SELECTOR, 'button[class="btn_sign_up"]').click()

# 로그인 상태를 유지함.


- 각 기업 별 <리뷰>와 <복지> 탭에 있는 정보들 크롤링하기
    - 리뷰 탭 크롤링 대상 
        - 1p : [전체 리뷰 평점, 복지 및 급여, 업무와 삶의 균형, 사내문화, 승진 기회 및 가능성, 경영진, 기업추천율, ceo 지지율, 성장 가능성] 
        - 모든 페이지 : 기업 리뷰들
    - 복지 탭 크롤링 대상
        - 1p : [전체 복지 평점, 복지 개수]
        - 모든 페이지 : 복지 리뷰들

## 최종 데이터 셋

- [판다스]   
    - 인덱스 -> jp_corp_code 기업순서   
    - 칼럼 ->   
전체 리뷰 평점, 복지 및 급여, 업무와 삶의 균형, 사내문화, 승진 기회 및 가능성, 경영진, 기업추천율,   
ceo 지지율, 성장 가능성 ,기업 리뷰들, 전체 복지 평점, 복지 개수 복지 리뷰들    
--> 기업 순서별로 각각의 리스트에 넣어주기

In [283]:
아이디, 회사순서, 전체리뷰평점,복지및급여,업무와삶의균형 ,사내문화 ,승진기회및가능성 ,경영진 ,기업추천율 ,ceo지지율 ,성장가능성, 총복지평점, 총복지개수, 장점리뷰, 단점리뷰 = [[] for _ in range(15)]

In [284]:
for i, corp_name in enumerate(jp_corp_code) :
    if corp_name != 'CJ대한통운' :
        # 1. 리뷰 탭에 있는 수치형 데이터 크롤링하기
        driver.get(f'https://www.jobplanet.co.kr/companies/{jp_corp_code[corp_name]}/reviews?page=1')
        
        리뷰평점 = float(driver.find_element(By.CSS_SELECTOR, 'span[class="rate_point"]').text)
        
        review_datas = driver.find_elements(By.CSS_SELECTOR, 'span[class="txt_point"]')
        review_composition = [float(review_data.text.replace('%', '').replace('--', '0')) if '%' in review_data.text else float(review_data.text) for review_data in review_datas]
        
        아이디.append(i+1)
        회사순서.append(corp_name)
        전체리뷰평점.append(리뷰평점)
        복지및급여.append(review_composition[0])
        업무와삶의균형.append(review_composition[1])
        사내문화.append(review_composition[2])
        승진기회및가능성.append(review_composition[3])
        경영진.append(review_composition[4])
        기업추천율.append(review_composition[5])
        ceo지지율.append(review_composition[6])
        성장가능성.append(review_composition[7])
        
        # 2. 복지 탭에 있는 수치형 데이터 크롤링하기
        driver.get(f'https://www.jobplanet.co.kr/companies/{jp_corp_code[corp_name]}/benefits')
        try : # 복지 종합 데이터가 존재할 경우
            복지평점 = driver.find_element(By.CSS_SELECTOR, 'span[class="welfare-star__point"]').text
            복지개수 = driver.find_element(By.CSS_SELECTOR, 'button[class="welfare-overview__num"]').text
            총복지평점.append(복지평점)
            총복지개수.append(복지개수)
        except :
            총복지평점.append('NUll')
            총복지개수.append('NUll')
        
        # 3. 리뷰 탭에 있는 장단점 텍스트 데이터 크롤링하기 
        concat_pos = '' 
        concat_neg = ''
        
        for page_num in range(1,10) :
            try : # dd[class="df1"]가 없을 수 있다. 즉 마지막 페이지까지 지났을 때 대비하기
                driver.get(f'https://www.jobplanet.co.kr/companies/{jp_corp_code[corp_name]}/reviews?page={page_num}&year%5B%5D=2024&year%5B%5D=2023&year%5B%5D=2022&year%5B%5D=2021')
                employee_talks = driver.find_elements(By.CSS_SELECTOR, 'dd[class="df1"]')

                
                pos = [posi.text.replace('\n', ' ').strip() for i, posi in enumerate(employee_talks) if i % 3 == 0 ]
                neg = [nega.text.replace('\n', ' ').strip() for i, nega in enumerate(employee_talks) if i % 3 == 1 ]


                concat_pos += '||'.join(pos) # '||'.join(pos) : 한 페이지에 대해서 장점을 모은 것
                concat_neg += '||'.join(neg) # concat_pos/neg : 모든 페이지의 장점과 단점을 모은 것 <- 장점/대상 리뷰 append 대상
            except :
                print(f'{corp_name}의 {page_num}에서 크롤링을 멈췄습니다. 페이지 범위가 아닌 다른 문제일 수 있으니 몇 개는 직접 확인해보세요')
        장점리뷰.append(concat_pos)
        단점리뷰.append(concat_neg)
            
    else : break
        
        


print('크롤링 완료')

    

크롤링 완료


In [285]:
jp_df = pd.DataFrame({'회사명' : 회사순서, '복지평점' : 총복지평점, '복지개수' : 총복지개수, '리뷰평점' : 전체리뷰평점, '복지 및 급여' : 복지및급여,
                      '워라밸' : 업무와삶의균형, '사내문화' : 사내문화, '승진기회및가능성' : 승진기회및가능성, '경영진 점수' : 경영진, '기업추천율' : 기업추천율,
                      'ceo지지율' : ceo지지율, '성장가능성' : 성장가능성, '직원들이 평가한 장점' : 장점리뷰, '직원들이 평가한 단점' : 단점리뷰 })

In [286]:
jp_df

,회사명,복지평점,복지개수,리뷰평점,복지 및 급여,워라밸,사내문화,승진기회및가능성,경영진 점수,기업추천율,ceo지지율,성장가능성,직원들이 평가한 장점,직원들이 평가한 단점
0,BGF리테일,3.0,57,3.0,3.0,2.9,2.8,2.8,2.5,41.0,46.0,30.0,일을 제대로 배워볼 수 있음 (넓고 깊게) 이외에 장점은 없음||연봉이 하는 일에 ...,장점을 뺀 모든게 단점 복지없음 사옥구림 이상한사람있음 친구들이 회사이름 못알아먹음...
1,BNK금융지주,2.0,3,2.8,3.0,2.7,2.7,2.5,2.0,33.0,33.0,0.0,그래도 복지는 생각보다 괜찮다 정도 딱히 장점 잘 모르겠음||금융회사의 장점. 복리...,경영진으로 이루어진 기업 ”까라면 까야지“가 실행되는 회사||큰 단점으로 말한 만한...
2,CJ,NUll,NUll,3.7,3.2,2.8,3.4,3.7,3.2,72.0,90.0,61.0,사내 복지와 물리적 환경 매우 좋음. 동료들의 수준 높음||워라밸은 챙길수있고 지주...,정적이고 무거운 분위기라 혁신적인 프로젝트가 추친력있게 진행되기 어려움||보고작업이...


### 임시 dd[class="df1"] 이거 넣어서 에러나면 break 하는 걸로 break 하는 걸로 해보자.

In [206]:
# # 24,23,22,21년도 리뷰들만 크롤링하기
# for page in range(1,500) :
    
    

In [233]:
driver.get('https://www.jobplanet.co.kr/companies/20575/reviews?page=34&year%5B%5D=2024&year%5B%5D=2023&year%5B%5D=2022&year%5B%5D=2021')
employee_talks = driver.find_elements(By.CSS_SELECTOR, 'dd[class="df1"]')

concat_pos = '' 
concat_neg = ''
pos = [posi.text.replace('\n', ' ').strip() for i, posi in enumerate(employee_talks) if i % 3 == 0 ]
neg = [nega.text.replace('\n', ' ').strip() for i, nega in enumerate(employee_talks) if i % 3 == 1 ]


concat_pos += '||'.join(pos) # '||'.join(pos) : 한 페이지에 대해서 장점을 모은 것
concat_neg += '||'.join(neg) # concat_pos/neg : 모든 페이지의 장점과 단점을 모은 것 <- 장점/대상 리뷰 append 대상

In [235]:
type(concat_neg)


str

In [237]:
for i in range(10) :
    print(i)
    for j in range(2) :
        try :
            sdf
            print('이건 나올 수 없음')
        except :
            print('몇 번 나올까?')
            break

   

0
몇 번 나올까?
1
몇 번 나올까?
2
몇 번 나올까?
3
몇 번 나올까?
4
몇 번 나올까?
5
몇 번 나올까?
6
몇 번 나올까?
7
몇 번 나올까?
8
몇 번 나올까?
9
몇 번 나올까?


In [ ]:
for i, corp_name in enumerate(jp_corp_code) :
    if corp_name == 'CJ대한통운' :

In [258]:
for a in range(1,4) :
    print(a)
    if a == 2 :
        break

1
2


In [272]:
len(아이디),len(회사순서) ,len(전체리뷰평점),len(복지및급여),len(업무와삶의균형) ,len(사내문화) ,len(승진기회및가능성) ,len(경영진) ,len(기업추천율) ,len(ceo지지율) ,len(성장가능성),len(총복지평점) ,len(총복지개수) ,len(장점리뷰) ,len(단점리뷰)

(3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 27, 27)

In [273]:
단점리뷰

['일을 제대로 배워볼 수 있음 (넓고 깊게) 이외에 장점은 없음||연봉이 하는 일에 비해 많이 받는 편 대우도 괜찮은 편||구성원들이 나름? 괜찮다 지역 일부 고인물 장급들은 수준이 낮다||복지는 좋아요. 복지만. 근데 그것조차 제대로 누리지 못했어요.||얇고 넓게 업무를 경험할수 있다는 점이 장점이자 단점임',
 '일을 제대로 배워볼 수 있음 (넓고 깊게) 이외에 장점은 없음||연봉이 하는 일에 비해 많이 받는 편 대우도 괜찮은 편||구성원들이 나름? 괜찮다 지역 일부 고인물 장급들은 수준이 낮다||복지는 좋아요. 복지만. 근데 그것조차 제대로 누리지 못했어요.||얇고 넓게 업무를 경험할수 있다는 점이 장점이자 단점임동기가 많고 선임들도 좋은 분들이 많다. 대체적으로 분위기는 좋은편||생각보다 괜찮은 워라밸 물론 케바케 사바사 차이는 많이 남||초봉은 괜찮다. 유명 편의점은 다들 알고 있으므로 집에서도 좋아한다.||체계적이다. 몇 달에 한번씩 재고조사를 하며 비리가 없는지 잘 확인한다||자유롭습니다 연차든 뭐든 초봉이 높습니다 ㅅㅅ 화이팅이요',
 '일을 제대로 배워볼 수 있음 (넓고 깊게) 이외에 장점은 없음||연봉이 하는 일에 비해 많이 받는 편 대우도 괜찮은 편||구성원들이 나름? 괜찮다 지역 일부 고인물 장급들은 수준이 낮다||복지는 좋아요. 복지만. 근데 그것조차 제대로 누리지 못했어요.||얇고 넓게 업무를 경험할수 있다는 점이 장점이자 단점임동기가 많고 선임들도 좋은 분들이 많다. 대체적으로 분위기는 좋은편||생각보다 괜찮은 워라밸 물론 케바케 사바사 차이는 많이 남||초봉은 괜찮다. 유명 편의점은 다들 알고 있으므로 집에서도 좋아한다.||체계적이다. 몇 달에 한번씩 재고조사를 하며 비리가 없는지 잘 확인한다||자유롭습니다 연차든 뭐든 초봉이 높습니다 ㅅㅅ 화이팅이요소속 팀원들이 다들 사람이 좋았습니다 그 외 특별한 장점은 없네요||업계1위 자부심 , 초봉, 지속적 시스템 투자, 맨파워||영업 직무에 한해서 워라벨은 챙기기 나름이고 연봉이 높다|